# Imports

In [454]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# import seaborn as sns
import warnings

import joblib

import psycopg2 as ps

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, ConfusionMatrixDisplay

import os

In [455]:
PATH_IGNORE = 'ignore'
PATH_DATA = 'data'
FILE_DATA = os.path.join(PATH_DATA, 'BASE DE DADOS PEDE 2024 - DATATHON.xlsx')
FILE_OLD_DATA = os.path.join(PATH_DATA, 'PEDE_PASSOS_DATASET_FIAP_ANTIGO.xlsx')

In [456]:
dfs_year = {}

dfs_year[2022] = pd.read_excel(FILE_DATA, sheet_name=0)
dfs_year[2023] = pd.read_excel(FILE_DATA, sheet_name=1)
dfs_year[2024] = pd.read_excel(FILE_DATA, sheet_name=2)

# Dicionário

**INDE** - Índice de desenvolvimento educacional
- Ponderação entre os indicadores abaixo

**IAN** - Indicador de adequação de nível
- Registros administrativos
- Registra a defasagem da aprendizagem do aluno por meio da comparação da Fase atual do aluno da Associação, com a equivalência das Fases de ensido na Associação e a divisão dos anos escolares do ensino formal (por idade)
- Utiliza a fase atual, e a fase ideal conforme a idade.
- 10 (em fase), 5 (moderada) ou 2,5 (severa).

**IDA** - Indicador de desempenho acadêmico
- Notas Provas PM e Média geral Universitária
- Registra a proficiência do aluno nos exames padronizados de avaliação interna das disciplinas oferecidas pela Associação (os alunos das Fases 0 a 7). No caso dos alunos da Fase 8, esse indicador registra a nota média obtida pelos alunos em todas as disciplinas curriculares cursadas, nas respectivas instituições de ensino superior conveniadas ao programa de bolsas de estudo da Associação.
- Utiliza as notas internas da associação. Matemática + Português + Inglês / 3
- 0 a 10

**IEG** - Indicador de engajamento
- Registros de entrega de lição de casa e de voluntariado
- Mede o engajamento do aluno nas tarefas curriculares requeridas em cada uma das disciplinas oferecidas na Associação, e/ou seu engajamento em ações de voluntariado desenvolvidas pela Associação.
- Soma das pontuações das tarefas realizadas e registradas / Número de tarefas

**IAA** - Indicador de Autoavaliação
- Questionário de Autoavaliação individual
- Registra por meio de um questionário padronizado e adaptado às distintas faixas etárias dos alunoa a Associação, uma autoavaliação do aluno sobre como se sente consigo mesmo, sobre os estudos, sobre sua família, amigos e comunidade, e e sobre como se sente a respeito da Associação Passos Mágicos.
- Soma das pontuações das respostas do estudante / Número total de perguntas

**IPS** - Indicador Psicossocial
- Questionário individual de avaliação das psicólogas
- Avaliação da equipe de psicólogas para caracterizar o desenvolvimento do aluno nas suas interações familiares, no seu desenvolvimento emocional, comportamental e da sua socialização na vida comunitária. Esse indicador também caracteriza o tipo de atendimento psicológico oferecido pela Associação ao aluno.
- Avaliações feitas por **psicólogos**.
- Soma das pontuações / Número de avaliadores

**IPP** - Indicador Psicopedagógico
- Questionário individual de avaliação dos pedagogos e professores
- Avaliação da equipe de educadores e psicopedagogos para caracterizar o desenvolvimento cognitivo, emocional, comportamental e de socialização do aluno no seu processo de aprendizado dentro do Programa de Aceleração do Conhecimento, dos Programas Educacionais e das Atividades Culturais promovidas pela Associação.
- Soma das avaliações sobre aspectos **pedagógicos** / Número de avaliações

**IPV** - Indicador de Ponto de Virada
- Questionário Individual de avaliação dos pedagogos e professores
- Avaliação da equipe de educadores e psicopedagogos a respeito do desenvolvimento do aluno das aptidões necessárias para iniciar a transformação da sua vida por meio da Educação, avaliando a integração do aluno à Associação, o seu desenvolvimento emocional, e o seu potencial acadêmico.
- Análises longitudinais de progresso acadêmico, engajamento e desenvolvimento emocional

**Pedras** - Baseadas no INDE
- Quartzo: 3.0 < INDE < 6.1
- Ágata: 6.1 < INDE < 7.2
- Ametista: 7.2 < INDE < 8.2
- Topázio: 8.2 < INDE < 9.4

# Padronizando colunas
Algumas colunas estão nomeadas diferentemente entre anos. Vamos arrumar isso antes de proceder.

In [457]:
for year in dfs_year:
    print(dfs_year[year].columns)

Index(['RA', 'Fase', 'Turma', 'Nome', 'Ano nasc', 'Idade 22', 'Gênero',
       'Ano ingresso', 'Instituição de ensino', 'Pedra 20', 'Pedra 21',
       'Pedra 22', 'INDE 22', 'Cg', 'Cf', 'Ct', 'Nº Av', 'Avaliador1',
       'Rec Av1', 'Avaliador2', 'Rec Av2', 'Avaliador3', 'Rec Av3',
       'Avaliador4', 'Rec Av4', 'IAA', 'IEG', 'IPS', 'Rec Psicologia', 'IDA',
       'Matem', 'Portug', 'Inglês', 'Indicado', 'Atingiu PV', 'IPV', 'IAN',
       'Fase ideal', 'Defas', 'Destaque IEG', 'Destaque IDA', 'Destaque IPV'],
      dtype='object')
Index(['RA', 'Fase', 'INDE 2023', 'Pedra 2023', 'Turma', 'Nome Anonimizado',
       'Data de Nasc', 'Idade', 'Gênero', 'Ano ingresso',
       'Instituição de ensino', 'Pedra 20', 'Pedra 21', 'Pedra 22', 'Pedra 23',
       'INDE 22', 'INDE 23', 'Cg', 'Cf', 'Ct', 'Nº Av', 'Avaliador1',
       'Rec Av1', 'Avaliador2', 'Rec Av2', 'Avaliador3', 'Rec Av3',
       'Avaliador4', 'Rec Av4', 'IAA', 'IEG', 'IPS', 'IPP', 'Rec Psicologia',
       'IDA', 'Mat', 'Por', 'In

In [458]:
def check_column(
        df : pd.DataFrame,
        column : str
):
    ''' Checks if a column exists in a DataFrame '''
    if column in df.columns:
        return True
    else: 
        return False

In [459]:
def check_column_in_many(
        df_list : list,
        column : str,
        verbose : bool = False
):
    '''Checks if a column exists in all given DataFrames'''
    exists = True
    count = 0
    for df in df_list:
        column_exists = check_column(df, column)
        if column_exists != True:
            exists = False
            if verbose:
                print(f'Column does not exists in dataframe {count}')
        count += 1
    return exists

In [460]:
# Colunas que queremos no nossos dataframes padronizados
universal_columns = {
    'Nome Anonimizado':'nome_anonimizado',
    'Fase':'fase',
    'Idade':'idade',
    'Gênero':'genero',
    'Ano ingresso':'ano_ingresso',
    'IAN':'ian',
    'IDA':'ida',
    'IEG':'ieg',
    'IAA':'iaa',
    'IPS':'ips',
    'IPP':'ipp',
    'IPV':'ipv',
    # 'Atingiu PV',
    'Mat':'matematica',
    'Por':'portugues',
    'Ing':'ingles',
    'Instituição de ensino':'instituicao'
}

In [461]:
uneven_columns = []

for column in universal_columns.keys():
    exists_in_all = check_column_in_many(
        df_list = list(dfs_year.values()),
        column = column
    )

    if exists_in_all == False:
        uneven_columns.append(column)

uneven_columns

['Nome Anonimizado', 'Idade', 'IPP', 'Mat', 'Por', 'Ing']

In [462]:
dfs_year[2022] = dfs_year[2022].rename(
    {'Nome' : 'Nome Anonimizado',
     'Idade 22' : 'Idade',
     'Matem' : 'Mat',
     'Portug': 'Por',
     'Inglês' : 'Ing'},
     axis = 'columns',
     errors = 'raise'
)

In [463]:
uneven_columns = []

for column in universal_columns:
    # print(f'Checking {column}')
    exists_in_all = check_column_in_many(
        df_list = list(dfs_year.values()),
        column = column
    )

    if exists_in_all == False:
        uneven_columns.append(column)
    # print(exists_in_all)
    # print()
    
uneven_columns

['IPP']

In [464]:
# temporary
dfs_year[2022]['IPP'] = 0

## (TBA) Tratando valores

## Mergendo dados

### Padronizando nomes das colunas

In [465]:
for year in dfs_year.keys():
    dfs_year[year] = dfs_year[year].rename(
        universal_columns,
        axis = 'columns',
        errors = 'raise'
    )
    dfs_year[year] = dfs_year[year][universal_columns.values()]

In [466]:
fixed_columns = ['nome_anonimizado', 'genero', 'ano_ingresso'] # will be taken from the oldest available

yearly_columns = list(universal_columns.values())
for col in fixed_columns:
    yearly_columns.remove(col)
yearly_columns # will be registered per-year

['fase',
 'idade',
 'ian',
 'ida',
 'ieg',
 'iaa',
 'ips',
 'ipp',
 'ipv',
 'matematica',
 'portugues',
 'ingles',
 'instituicao']

In [467]:
data = {}

for year in dfs_year.keys():
    data[year] = dfs_year[year][['nome_anonimizado'] + yearly_columns].copy()
    data[year] = data[year].set_index('nome_anonimizado')
    data[year].columns = [f'{col}_{year}' for col in data[year].columns if col != 'nome_anonimizado']

### Atualizando alunos de 2022 até 2024

In [ ]:
df_2022 = dfs_year[2022][fixed_columns].copy()
df_2022 = df_2022.set_index('nome_anonimizado')

for year in dfs_year.keys():
    df_2022 = pd.merge(
        left = df_2022,
        right = data[year],
        on = 'nome_anonimizado',
        how = 'left'
    )
df_2022.tail(3)

,genero,ano_ingresso,fase_2022,idade_2022,ian_2022,ida_2022,ieg_2022,iaa_2022,ips_2022,ipp_2022,...,ida_2024,ieg_2024,iaa_2024,ips_2024,ipp_2024,ipv_2024,matematica_2024,portugues_2024,ingles_2024,instituicao_2024
nome_anonimizado,,,,,,,,,,,,,,,,,,,,,
Aluno-858,Menino,2022,0,7,10.0,7.7,7.1,10.0,5.6,0,...,8.25,6.833333,9.002,2.51,7.500000,7.500000,10.0,6.5,NaN,Pública
Aluno-859,Menina,2022,0,7,10.0,8.9,8.7,8.5,7.5,0,...,8.00,9.337121,8.002,7.51,8.958333,8.336667,8.0,8.0,NaN,Pública
Aluno-860,Menino,2022,0,7,10.0,8.1,8.5,9.5,7.5,0,...,8.50,9.791667,9.002,7.51,8.750000,7.946667,8.0,9.0,NaN,Pública


### Atualizando alunos de 2023 até 2024

In [476]:
df_2023 = dfs_year[2023][fixed_columns].copy()
df_2023 = df_2023.set_index('nome_anonimizado')

old_students = df_2022.index.to_list()
current_students = df_2023.index.to_list()

new_students = list(set(current_students) - set(old_students))

df_2023 = df_2023.loc[new_students]
df_2023.head(3)

,genero,ano_ingresso
nome_anonimizado,,
Aluno-956,Masculino,2023
Aluno-1132,Masculino,2023
Aluno-1090,Masculino,2023


In [477]:
for year in dfs_year.keys():
    df_2023 = pd.merge(
        left = df_2023,
        right = data[year],
        on = 'nome_anonimizado',
        how = 'left'
    )
df_2023.tail(3)

,genero,ano_ingresso,fase_2022,idade_2022,ian_2022,ida_2022,ieg_2022,iaa_2022,ips_2022,ipp_2022,...,ida_2024,ieg_2024,iaa_2024,ips_2024,ipp_2024,ipv_2024,matematica_2024,portugues_2024,ingles_2024,instituicao_2024
nome_anonimizado,,,,,,,,,,,,,,,,,,,,,
Aluno-1031,Masculino,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.25,8.636364,9.502,7.51,5.3125,5.83,5.5,7.0,NaN,Pública
Aluno-864,Feminino,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aluno-888,Masculino,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.50,10.000000,9.002,6.88,7.5000,8.28,9.0,8.0,NaN,Pública


---

# 1. Adequação do nível (IAN): Qual é o perfil geral de defasagem dos alunos (IAN) e como ele evolui ao longo do ano?
(Ex.: quantos alunos estão moderadamente ou severamente defasados?)

# 2. Desempenho acadêmico (IDA): O desempenho acadêmico médio (IDA) está melhorando, estagnado ou caindo ao longo das fases e anos?

# 3. Engajamento nas atividades (IEG): O grau de engajamento dos alunos (IEG) tem relação direta com seus indicadores de desempenho (IDA) e do ponto de virada (IPV)?

# 4. Autoavaliação (IAA): As percepções dos alunos sobre si mesmos (IAA) são coerentes com seu desempenho real (IDA) e engajamento (IEG)?

# 5. Aspectos psicossociais (IPS): Há padrões psicossociais (IPS) que antecedem quedas de desempenho acadêmico ou de engajamento?

# 6. Aspectos psicopedagógicos (IPP): As avaliações psicopedagógicas (IPP) confirmam ou contradizem a defasagem identificada pelo IAN?

# 7. Ponto de virada (IPV): Quais comportamentos - acadêmicos, emocionais ou de engajamento - mais influenciam o IPV ao longo do tempo?

# 8. Multidimensionalidade dos indicadores: Quais combinações de indicadores (IDA + IEG + IPS + IPP) melhor explicam o desempenho global do aluno (INDE)?

# 9. Previsão de risco com Machine Learning: Quais padrões nos indicadores permitem identificar alunos em risco antes de queda no desempenho ou aumento da defasagem? Construa um modelo preditivo que mostre uma probabilidade do aluno ou aluna entrar em risco de defasagem.

# 10. Efetividade do programa: Os indicadores mostram melhora consistente ao longo do ciclo nas diferentes fases (Quartzo, Ágata, Ametista e Topázio), confirmando o impacto real do programa?

# 11. Insights e criatividade: Você pode adicionar mais insights e pontos de vista não abordados nas perguntas, utilize a criatividade e a análise dos dados para trazer sugestões para a Passos Mágicos.

# Quais padrões nos indicadores permitem identificar alunos em risco antes de queda no desempenho ou aumento da defasagem?